In [ ]:
import re
import time
import numpy as np
import pathlib
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import psycopg2
from urllib.request import urlretrieve
from sqlalchemy import create_engine
import json

In [ ]:
# Paths
INPUT_ROOT = pathlib.Path().cwd()/'source'
WORKING_ROOT = pathlib.Path().cwd()/'working'
OUTPUT_ROOT = pathlib.Path().cwd()/'output'
MODELS_PATH = pathlib.Path().cwd()/'models'

for path in [INPUT_ROOT, WORKING_ROOT, OUTPUT_ROOT, MODELS_PATH]:
    path.mkdir(exist_ok=True)

# Read config file
with open('config.json') as f:
    config = json.load(f)

# Input files
SOURCE_PATH = INPUT_ROOT/config["paths"]["source_path"]
TARGET_PATH = INPUT_ROOT/config["paths"]["target_path"]

# Size constants
BATCH_SIZE = config["execution"]["batch_size"]
NUM_TOP_MATCHES = config["execution"]["num_top_matches"]

# Model links
BIOWORDVEC_LINK=config["models"]["BioWordVec"]
SPACY_MODEL_LINK=config["models"]["Spacy_SciSM"]

In [ ]:
# Loading the biomedical language model
model_name = SPACY_MODEL_LINK.split("/")[-1]
arch_model_path = MODELS_PATH/model_name
if not arch_model_path.exists():
    urlretrieve(SPACY_MODEL_LINK, arch_model_path)
%pip install $arch_model_path

In [ ]:

nlp = spacy.load(model_name.split("-")[0])

In [ ]:
# Read the CSV files into dataframes
voc1_df = pd.read_csv(SOURCE_PATH)
voc2_df = pd.read_csv(TARGET_PATH)
voc1_df=voc1_df[:10] # следующая 2000:4000
voc1_df = voc1_df.reset_index(drop=True)

In [ ]:
voc1_df

In [ ]:
voc2_df

In [ ]:
# # words to remove from strings
# stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
#              'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
#              'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what',
#              'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were',
#              'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
#              'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
#              'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from',
#              'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
#              'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
#              'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
#              'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain',
#              'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
#              "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
#              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't",
#              'wouldn', "wouldn't",
#               # don't care  about right and left most of the time
#              #'right','left']

In [ ]:
# Stop_words that were received from SciSpacy model (UMLS based)
stop_words = ["'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again',
              'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 
              'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 
              'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 
              'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 
              'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 
              'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 
              'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 
              'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 
              'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 
              'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 
              'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 
              'last', 'latter', 'latterly', 'least', 'less', 'made', 'make', 'many', 'may', 'me', 'meanwhile', 
              'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', "n't", 
              'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone',
              'nor', 'not', 'nothing', 'now', 'nowhere', 'n‘t', 'n’t', 'of', 'off', 'often', 'on', 'once', 'one', 
              'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
              'part', 'per', 'perhaps', 'please', 'put', 'quite', 'rather', 're', 'really', 'regarding', 'same',
              'say', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show',
              'side', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'something', 'sometime',
              'sometimes', 'somewhere', 'still', 'such', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 
              'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon',
              'these', 'they', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 
              'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 
              'until', 'up', 'upon', 'us', 'used', 'using', 'various', 'very', 'via', 'was', 'we', 'well', 'were', 
              'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
              'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole',
              'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours',
              'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', '’s', '’ve']

# aditional task-specific stop_words

stop_words = stop_words + ['noc', 'nos', '[d]', 'unknown_unit', '|', 'see comment', 'due', 'nec', 'unspecified', '[v]', '(see comments)',
              '(disorder)', '(procedure)', '(finding)', '(observable entity)']

In [ ]:
# Convert the titles to lowercase
voc1_df['concept_name_processed'] = voc1_df['concept_name'].str.lower()
voc2_df['concept_name_processed'] = voc2_df['concept_name'].str.lower()


# Remove stopwords
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())))


# ignores weird symbols
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: re.sub(r"\s+", " ", str(x).encode('ascii', 'ignore').decode()) if isinstance(x, str) else str(x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: re.sub(r"\s+", " ", str(x).encode('ascii', 'ignore').decode()) if isinstance(x, str) else str(x))

# replace '-' with ' '
#voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].map(lambda x: re.sub('-', ' ', x))
#voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].map(lambda x: re.sub('-', ' ', x))

# replace '/' with ' '
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].str.replace('/', ' ')
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].str.replace('/', ' ')


# Remove punctuation, digits etc
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].str.replace(r'[^\w\s]','')
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].str.replace(r'[^\w\s]','')


In [ ]:
voc1_df.head(5)

In [ ]:
voc2_df.head(5)

In [ ]:
def tokenize_with_spacy(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
    return tokens

start_time = time.time()

voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(tokenize_with_spacy)
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(tokenize_with_spacy)

elapsed_time = (time.time() - start_time)/60
print(f"Total elapsed time for tokenization: {elapsed_time:.1f} minutes")


def lemmatize_with_spacy(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

start_time = time.time()

voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: lemmatize_with_spacy(' '.join(x)))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: lemmatize_with_spacy(' '.join(x)))

elapsed_time = (time.time() - start_time)/60
print(f"Total elapsed time for lemmatization: {elapsed_time:.1f} minutes")




In [ ]:
voc1_df.head()

In [ ]:
voc2_df.head()

In [ ]:
# Save previously prepared vocabulary for future use
voc2_df.to_csv(WORKING_ROOT / (TARGET_PATH.stem + '_processed.csv'), index=False)

# TfIDF

In [ ]:
# Preprocess and compute the TF-IDF document-term matrix for the snomed concepts
voc2_docs = voc2_df['concept_name_processed'].apply(lambda x: ''.join(x)).tolist()
tfidf_vectorizer = TfidfVectorizer()
voc2_dtm = tfidf_vectorizer.fit_transform(voc2_docs)

# Calculate cosine similarities and find most similar snomed concepts for each meddra concept
results = []

start_time = time.time()
processed_count = 0

for voc1_index, voc1_row in voc1_df.iterrows():
    voc1_tokens = voc1_row['concept_name_processed']
    voc1_name = voc1_row['concept_name']
    voc1_concept_id = voc1_row['concept_id']

    voc1_text = ' '.join(voc1_tokens)
    voc1_dtm = tfidf_vectorizer.transform([voc1_text])

    similarities = cosine_similarity(voc1_dtm, voc2_dtm)[0]

    # Find indices of most similar voc2 (SNOMED and etc.) concepts
    top_indices = np.argsort(similarities)[-NUM_TOP_MATCHES:][::-1]

    for max_index in top_indices:
        voc2_concept_id = voc2_df.loc[max_index, 'concept_id']
        voc2_name = voc2_df.loc[max_index, 'concept_name']
        similarity = similarities[max_index]

        # Add a separate line of information to the results
        results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, similarity))

    processed_count += 1
    if processed_count % BATCH_SIZE == 0:
        elapsed_time = (time.time() - start_time) / 60
        print(f"Processed {processed_count} concepts in {elapsed_time:.1f} minutes")

# Sort the results by similarity in descending order
results.sort(key=lambda x: x[4], reverse=True)

# Convert results to DataFrame
columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'similarity']
results_df = pd.DataFrame(results, columns=columns)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")

In [ ]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'similarity'], ascending=False)
df_sorted['origin']='TfIDF'
df_sorted.to_csv(WORKING_ROOT/('final_table_tfidf.csv'), index=False)
df_sorted.head(50)

In [ ]:
df_tfidf=df_sorted

# Fuzz_ratio

**Rapidfuzz.fuzz.ratio** is a function from the rapidfuzz library in Python, which is used for string matching and similarity comparisons. The rapidfuzz library is known for its efficient and fast computation of string similarities, often utilized in scenarios where large datasets require rapid processing of string comparison tasks.

The fuzz.ratio function computes the similarity between two strings, returning a score that represents how similar the two strings are. The score is an integer ranging from 0 to 100, where 100 indicates an exact match (the strings are identical), and lower scores indicate less similarity.

This function implements an algorithm similar to the Levenshtein Distance, which calculates the number of single-character edits (insertions, deletions, or substitutions) required to change one word into the other. However, rapidfuzz has optimized the computation for performance, making it faster than many other implementations of similar algorithms.

In summary, rapidfuzz.fuzz.ratio is a fast and efficient way to compare two strings and quantify their similarity, widely used in tasks like data cleaning, deduplication, and matching in various applications.

In [ ]:
#import Levenshtein as lev

In [ ]:
voc1_df['concept_name_processed'] = voc1_df['concept_name_processed'].apply(lambda x: sorted(x))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
voc2_df['concept_name_processed'] = voc2_df['concept_name_processed'].apply(lambda x: sorted(x))

In [ ]:
voc2_df.head(10)

In [ ]:
# # Old variant of Levenshtein
# start_time = time.time()
# results = []

# count = 0

# for voc1_index, voc1_row in voc1_df.iterrows():
#     voc1_name = voc1_row['concept_name']
#     voc1_concept_id=voc1_row['concept_id']
#     voc1_tokens = voc1_row['concept_name_processed']

#     min_distances = [float('inf')] * NUM_TOP_MATCHES  # Список для хранения наименьших расстояний
#     top_matches = [[] for _ in range(NUM_TOP_MATCHES)]  # Список для хранения ближайших сочетаний

#     for voc2_index, voc2_row in voc2_df.iterrows():
#         voc2_concept_id = voc2_row['concept_id']
#         voc2_name = voc2_row['concept_name_processed']

#         # Calculate Levenshtein distance between voc1 and voc2 concept names
#         distance = lev.distance(''.join(voc1_tokens), ''.join(voc2_name))

#         # Обновляем ближайшие совпадения, если найдено более близкое совпадение
#         for i in range(NUM_TOP_MATCHES):
#             if distance < min_distances[i]:
#                 min_distances.insert(i, distance)
#                 min_distances.pop()
#                 top_matches.insert(i, (voc2_concept_id, voc2_row['concept_name'], distance))
#                 top_matches.pop()
#                 break

#     # Сортируем top_matches по возрастанию min_distance перед добавлением в results
#     top_matches.sort(key=lambda x: x[2])

#     # Добавляем ближайшие совпадения в результаты
#     for matches in top_matches:
#         if matches:  # Проверяем, что список не пустой
#             voc2_concept_id, voc2_name, min_distance = matches
#             results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, min_distance))

#     count += 1
#     if count % 10 == 0:
#         elapsed_time = (time.time() - start_time) / 60
#         print(f"Elapsed time: {elapsed_time:.1f} minutes for {count} концептов")

# # Convert results to DataFrame
# columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'min_distance']
# results_df = pd.DataFrame(results, columns=columns)

# elapsed_time = (time.time() - start_time) / 60
# print(f"Total elapsed time: {elapsed_time:.1f} minutes")

In [ ]:
# # Output the results as a Pandas dataframe
# df_sorted = results_df.sort_values(by=['voc1_name', 'min_distance'], ascending=True)
# df_sorted['origin']='Levenshtein_distance'
# df_sorted.to_csv('final_table'+'_levenshtein_old.csv', index=False)
# df_sorted.head(50)

In [ ]:
import rapidfuzz

In [ ]:
# new version from ChatGPT
import time
import pandas as pd
import numpy as np
from rapidfuzz import process
import concurrent.futures

start_time = time.time()

# Preprocessing
voc1_df['voc1_tokens_str'] = voc1_df['concept_name_processed'].apply(''.join)
voc2_names = voc2_df['concept_name_processed'].apply(''.join).tolist()

# Define a function to process a chunk of rows
def process_chunk(chunk, voc2_names):
    results = []
    for _, voc1_row in chunk.iterrows():
        voc1_name = voc1_row['concept_name']
        voc1_concept_id = voc1_row['concept_id']
        voc1_tokens_str = voc1_row['voc1_tokens_str']

        # Get top matches using rapidfuzz
        top_matches = process.extract(voc1_tokens_str, voc2_names, limit=NUM_TOP_MATCHES, scorer=rapidfuzz.fuzz.ratio)

        # Unpack the results correctly
        for match in top_matches:
            matched_string, score, index = match
            voc2_concept_id = voc2_df.iloc[index]['concept_id']
            voc2_name = voc2_df.iloc[index]['concept_name']
            results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, score))

    return results

# Split DataFrame into chunks for parallel processing
chunks = [voc1_df.iloc[i:i + BATCH_SIZE] for i in range(0, voc1_df.shape[0], BATCH_SIZE)]

# Parallel processing of chunks
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_chunk, chunks, [voc2_names] * len(chunks)))

# Flatten the list of results
flattened_results = [item for sublist in results for item in sublist]

# Convert results to DataFrame
columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'fuzz_ration']
results_df = pd.DataFrame(flattened_results, columns=columns)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")



In [ ]:
results_df

In [ ]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'fuzz_ratio'], ascending=False)
df_sorted['origin']='fuzz_ratio'
df_sorted.to_csv(WORKING_ROOT / 'final_table_fuzz_ratio.csv', index=False)
df_sorted.head(50)

In [ ]:
df_fuzz_ratio=df_sorted

# BioWordVec

In [ ]:
# Test if the processed_name contains a list object
# We need a sorted list
for voc_df in (voc1_df, voc2_df):
    if isinstance(voc_df.loc[voc_df.index[0], 'concept_name_processed'], list):
        voc_df['concept_name_processed'] = voc_df['concept_name_processed'].apply(sorted)
    else:
        voc_df['concept_name_processed'] = voc_df['concept_name_processed'].apply(lambda x: sorted(x.strip("[]").replace("'", "").split(', ')))


In [ ]:
import time
import numpy as np
import gensim
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained word embeddings model
model_path = MODELS_PATH / 'BioWordVec_PubMed_MIMICIII_d200.vec.bin'
if not model_path.exists():
    print(f"Model is not found in {str(model_path)}. Downloading...")
    urlretrieve(BIOWORDVEC_LINK, model_path)

word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
start_time = time.time()

# Функция для векторизации текста с использованием BioWordVec
def vectorize_text(text, model):
    tokens = text.split()
    vectors = [model[token] for token in tokens if token in model]
    if vectors:
        return np.mean(vectors, axis=0)  # Усреднение векторов
    else:
        return np.zeros(model.vector_size)  # Возвращает нулевой вектор, если нет известных слов

# Применение функции к медицинским терминам в обоих наборах данных
voc1_df['voc1_vector'] = voc1_df['concept_name_processed'].apply(lambda x: vectorize_text(' '.join(x), word2vec_model))
voc2_df['voc2_vector'] = voc2_df['concept_name_processed'].apply(lambda x: vectorize_text(' '.join(x), word2vec_model))

# Удаление строк с отсутствующими векторами
voc1_df = voc1_df.dropna(subset=['voc1_vector'])
voc2_df = voc2_df.dropna(subset=['voc2_vector'])


In [ ]:
start_time = time.time()

v1 = np.vstack(voc1_df['voc1_vector'])
v2 = np.vstack(voc2_df['voc2_vector'])
similarity = cosine_similarity(v1, v2)
tops = (-similarity).argsort()[:, :NUM_TOP_MATCHES]

results_df = pd.concat(     # concat (axis = 0) the list of data frames
    [pd.concat( # generate a list of data frames by concatenating (axis = 1) selected rows and cols from voc1 and voc2
        [voc1_df.loc[[i], ['concept_name', 'concept_id']].reset_index(drop=True),  # for each target concept from voc_1
         voc2_df.loc[[t], ['concept_id', 'concept_name']].reset_index(drop=True),        # add potential mapping match from voc_2
         pd.Series(similarity[i][t], name='Similarity')], axis= 1)                       # and similarity value
     for i in range(0, len(voc1_df)) for t in tops[i]]                                   # `i` - target vocabulary index, `t` - index in top similarity array
).reset_index(drop=True)

elapsed_time = (time.time() - start_time) / 60
print(f"Total elapsed time: {elapsed_time:.1f} minutes")

In [ ]:
# Old working but very slow code

# results = []

# for voc1_index, voc1_row in voc1_df.iterrows():
#     voc1_name = voc1_row['concept_name']
#     voc1_vector = voc1_row['voc1_vector']
#     voc1_concept_id = voc1_row['concept_id']

#     similarities = []
#     for voc2_index, voc2_row in voc2_df.iterrows():
#         voc2_name = voc2_row['concept_name']  # Сохраняем concept_id
#         voc2_concept_id = voc2_row['concept_id']
#         voc2_vector = voc2_row['voc2_vector']

#         if voc1_vector is not None and voc2_vector is not None:
#             similarity = cosine_similarity([voc1_vector], [voc2_vector])[0][0]
#             similarities.append((voc2_name, voc2_concept_id, similarity))

#     similarities.sort(key=lambda x: x[2], reverse=True)
#     top_matches = similarities[:NUM_TOP_MATCHES]

#     for voc2_name, voc2_concept_id, similarity in top_matches:
#         results.append((voc1_name, voc1_concept_id, voc2_concept_id, voc2_name, similarity))  # Добавляем voc1_index и voc2_name

#     # Счетчик каждых 10 обработанных строк
#     if (voc1_index + 1) % 10 == 0:
#         elapsed_time = (time.time() - start_time) / 60
#         print(f"Elapsed time: {elapsed_time:.1f} minutes for processed {voc1_index + 1} rows")

        
# # Создать DataFrame с результатами
# columns = ['voc1_name', 'voc1_concept_id', 'voc2_concept_id', 'voc2_name', 'similarity']  # Добавляем 'voc1_index'
# results_df = pd.DataFrame(results, columns=columns)

# elapsed_time = (time.time() - start_time) / 60
# print(f"Total elapsed time: {elapsed_time:.1f} minutes")

In [ ]:
results_df.head(50)

In [ ]:
# Modification to new variant
new_columns = list(results_df.columns)
new_columns[0] = 'voc1_name'
new_columns[1]='voc1_concept_id'
new_columns[2]='voc2_concept_id'
new_columns[3]='voc2_name'
new_columns[4]='similarity'
results_df.columns = new_columns


# Присвойте новый список названий столбцов атрибуту 'columns' DataFrame
#results_df['voc1_concept_id']=0
results_df

In [ ]:
# Output the results as a Pandas dataframe
df_sorted = results_df.sort_values(by=['voc1_name', 'similarity'], ascending=False)
df_sorted['origin']='BioWordVec'
df_sorted.to_csv(WORKING_ROOT/'final_table_biowordvec.csv', index=False)
df_sorted.head(50)

In [ ]:
df_biowordvec=df_sorted

# Final merge tables and remove duplicates

In [ ]:
import datetime
current_datetime = datetime.datetime.now()
current_date = current_datetime.date()

In [ ]:
# Concatenate all results
loaded = []
for table in WORKING_ROOT.glob("final_*.csv"):
    print(f"Processing {table.name}...")
    df = pd.read_csv(table)
    df.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'], inplace=True)
    loaded.append(df)

In [ ]:
# Объединить таблицы, устранить дублирующие строки с одинаковыми парами сорс-код и potential_concept_id
table_name = 'combined_table_'+str(current_date)+'.csv'
df_exp = pd.concat(loaded)
df_exp = df_exp.sort_values(by=['voc1_name']) 
df_exp

In [ ]:
# Группировка и создание поля flag
grouped = df_exp.groupby(['voc1_name', 'voc2_concept_id'])

def combine_origin(group):
    if len(group) > 1:
        group['flag'] = ' / '.join(sorted(group['origin']))
        return group.iloc[:1]  # оставляем только одну строку для объединенных
    else:
        group['flag'] = group['origin'].iloc[0]
        return group

df_exp = grouped.apply(combine_origin).reset_index(drop=True)
df_exp.drop(['origin'], axis=1, inplace=True)

# Вывод результата
df_exp

In [ ]:
result = df_exp.groupby(['flag']).size().sort_values(ascending=False).reset_index(name='count')
result

In [ ]:
df_exp = df_exp.drop_duplicates(subset=['voc1_name', 'voc2_concept_id'])
df_exp.to_excel('my_table_30112023.xlsx', index=False)

In [ ]:
df_exp = pd.read_excel('my_table_30112023.xlsx')
df_exp

In [ ]:
#To download table into database scheme 
login="dev_test5"
passw = "7hGg365$%fhhTfr$dJ"
engine = create_engine('postgresql://{}:{}@ovh07.odysseusinc.com:5555/postgres'.format(login, passw))
table_name = 'am_gpt_meddra_pt_first_2000_051223_out_75_full_variant'
df_exp.to_sql(table_name, engine, if_exists='replace', index=False)
print(f'Import {table_name} was sucessful!')

In [ ]:
combined_df = df_exp.copy()
combined_df['question'] = 'Pick the best semantic match for '
combined_df = combined_df[['question'] + [col for col in combined_df.columns if col != 'question']]
combined_df.rename(columns={'voc1_name': 'source_code_description', 'voc2_concept_id': 'potential_target_concept_id'}, inplace=True)
combined_df = combined_df.sort_values(by=['source_code_description', 'potential_target_concept_id'])
combined_df = combined_df[['question', 'source_code_description', 'potential_target_concept_id']]
combined_df['chatgptreply']=''
combined_df['target_concept_id'] = None
combined_df['log_id']=None
combined_df['target_concept_id'] = combined_df['target_concept_id'].astype('Int64')
combined_df['log_id'] = combined_df['log_id'].astype('Int64')
combined_df

In [ ]:
#To download table into database scheme 
login="dev_test5"
passw = "7hGg365$%fhhTfr$dJ"
engine = create_engine('postgresql://{}:{}@ovh07.odysseusinc.com:5555/postgres'.format(login, passw))
table_name = 'am_gpt_meddra_pt_first_2000_051223_out_75'
combined_df.to_sql(table_name, engine, if_exists='replace', index=False)
print(f'Import {table_name} was sucessful!')